In [ ]:
import pandas_datareader as pdr

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv(filepath_or_buffer = 'https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv')
print('Data Shape:', df.shape)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df2=df.reset_index()['Close']

In [ ]:
df2.shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df2=scaler.fit_transform(np.array(df2).reshape(-1,1))

In [ ]:
df2.shape

In [ ]:
df2

In [ ]:
training_size=int(len(df2)*0.65)
test_size=len(df2)-training_size
train_x,test_x=df2[0:training_size,:],df2[training_size:len(df2),:1]

In [ ]:
training_size,test_size

In [ ]:
import numpy as np
def create_dataset(dataset,time_step=1):
  dataX,dataY=[],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX),np.array(dataY)

In [ ]:
time_step=100
X_train,y_train=create_dataset(train_x,time_step)
X_test,y_test=create_dataset(test_x,time_step)

In [ ]:
print(X_train)

In [ ]:
print(X_test.shape),print(y_test.shape)

In [ ]:
print(X_train.shape),print(y_train.shape)

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM 

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:

math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
look_back=100
trainPredictPlot=np.empty_like(df2)
trainPredictPlot[:,:]=np.nan
trainPredictPlot[look_back:len(train_predict)+look_back,:]=train_predict


testPredictPlot=np.empty_like(df2)
testPredictPlot[:,:]=np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df2)-1,:]=test_predict

plt.plot(scaler.inverse_transform(df2))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()


In [ ]:
len(test_x)

In [ ]:
713*30

In [ ]:
x_input=test_x[613:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input

In [ ]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):

  if(len(temp_input)>100):
    x_input=np.array(temp_input [1:])
    print("{} day input{}".format(i,x_input))
    x_input=x_input.reshape(1,-1)
    x_input= x_input.reshape((1, n_steps, 1))

    yhat =model.predict(x_input, verbose=0)

    print("{} day output{}".format(i, yhat))
    temp_input.extend(yhat[0].tolist())
    temp_input=temp_input[1:]

    lst_output.extend(yhat.tolist())
    i=i+1

  else:
    x_input = x_input.reshape((1, n_steps,1))
    yhat=model.predict(x_input, verbose =0)
    print(yhat[0])
    temp_input.extend(yhat[0].tolist())
    print(len(temp_input))
    lst_output.extend(yhat.tolist())
    i=i+1

print(lst_output)



In [ ]:
new_day=np.arange(1,101)
pred_day=np.arange(101,131)


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
len(df2)

In [ ]:
df1=df2.tolist()
df1.extend(lst_output)

In [ ]:
plt.plot(new_day,scaler.inverse_transform(df2[1935:]))
plt.plot(pred_day,scaler.inverse_transform(lst_output))

In [ ]:
df1=df2.tolist()
df1.extend(lst_output)
plt.plot(df1[1000:])